In [95]:
import bs4
import requests
import pandas as pd

Scrape beaches page from rhodes.gr

In [17]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get('https://www.rhodes.gr/tourist-guide/beaches/?lang=en', headers=headers)
response

<Response [200]>

In [100]:
x = [1, 2, 4, 6]
g = iter(x)
for _ in g:
    print(next(g))

2
6


In [19]:
soup = bs4.BeautifulSoup(response.text, "html.parser")

In [102]:
main_content = soup.find_all("div", {"class": "td-main-content-wrap"})[0]
beaches_text = main_content.find_all("div", {"class": "txt_page"})[0]

data = {"name": [], "photo_url": [], "description": []}
beaches_gen = iter(beaches_text.find_all("p"))

for entry in beaches_gen:
    text = entry.get_text()
    
    if not text:
        continue
    
    if text[0].isdigit():
        name = text.split(".")[1][1:]
        data["name"].append(name)
        data["description"].append(next(beaches_gen).get_text())
                
        photos = entry.find_all("img")
        if photos:
            data["photo_url"].append(photos[0]["src"])
        else:
            print("Failed to find photo for {}.".format(name))
            data["photo_url"].append(None)

Failed to find photo for Fournoi.
Failed to find photo for Gournes.
Failed to find photo for Haraki.
Failed to find photo for Ialyssos.
Failed to find photo for Kiotari.
Failed to find photo for Lardos.
Failed to find photo for Vagies.


In [103]:
data_df = pd.DataFrame(data)
data_df

,name,photo_url,description
0,Afandou,http://www.rhodes.gr/wp-content/uploads/2016/0...,"Great in length, with small pebbles and sand a..."
1,Agathi,http://www.rhodes.gr/wp-content/uploads/2016/0...,"Sandy beach, sheltered. Located east of Rhodes..."
2,Anthony Quinn,http://www.rhodes.gr/wp-content/uploads/2016/0...,Ideal for scuba diving because of the underwat...
3,Faliraki,http://www.rhodes.gr/wp-content/uploads/2016/0...,Four kilometers of length one of the most orga...
4,Fournoi,None,Small beach near the village of Monolith with ...
5,Gennadi,http://www.rhodes.gr/wp-content/uploads/2016/0...,Long beach with sand and small pebbles. It has...
6,Glystra,http://www.rhodes.gr/wp-content/uploads/2016/0...,Beach with fine sand. Quiet beach it has umbre...
7,Gournes,None,"Cosmopolitan beach on the east coast, with umb..."
8,Haraki,None,Beach is visited mainly by families. With pebb...
9,Ialyssos,None,It is one of the most organized of the island ...


In [104]:
data_df.to_csv("beaches.csv", index=False)

Find google maps link by searching

In [152]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}

def search_name(name):
    sname = "+".join(name.split(" "))
    return "https://www.google.gr/maps/search/{}".format(sname)


def find_link(text, target="@36"):
    idx = text.find(target)
    start = idx
    while text[start] != '"':
        start -= 1
    end = idx
    while text[end] != '"':
        end += 1
    return text[start + 1:end]


def get_link(name):
    sname = search_name(name + " beach")
    response = requests.get(sname, headers=headers)
    link = find_link(response.text)
    if link:
        return link
    print("Failed to find link for {}.".format(sname))
    
    sname = search_name(name)
    response = requests.get(sname, headers=headers)
    link = find_link(response.text)
    if link:
        return link
    print("Failed to find link for {}.".format(sname))
        
    return None

In [153]:
data_df["google_maps"] = data_df["name"].map(get_link)

Failed to find link for https://www.google.gr/maps/search/Afandou+beach.
Failed to find link for https://www.google.gr/maps/search/Faliraki+beach.
Failed to find link for https://www.google.gr/maps/search/Gennadi+beach.
Failed to find link for https://www.google.gr/maps/search/Gournes+beach.
Failed to find link for https://www.google.gr/maps/search/Gournes.
Failed to find link for https://www.google.gr/maps/search/Ialyssos+beach.
Failed to find link for https://www.google.gr/maps/search/Kalathos+beach.
Failed to find link for https://www.google.gr/maps/search/Kallithea+beach.
Failed to find link for https://www.google.gr/maps/search/Kallithea.
Failed to find link for https://www.google.gr/maps/search/Kiotari+beach.
Failed to find link for https://www.google.gr/maps/search/Lardos+beach.
Failed to find link for https://www.google.gr/maps/search/Lindos+beach.
Failed to find link for https://www.google.gr/maps/search/Pefkoi+beach.
Failed to find link for https://www.google.gr/maps/search/P

In [155]:
for x, y in zip(data_df["name"], data_df["google_maps"]):
    print(x)
    print(y)
    print("\n\n")

Afandou
https://www.google.gr/maps/preview/place/%CE%91%CF%86%CE%AC%CE%BD%CF%84%CE%BF%CF%85+851+03/@36.2944144,28.1620173,3216a,13.1y/data\\u003d!4m2!3m1!1s0x149571d828fd0987:0x400bd2ce2b9b180\



Agathi
https://www.google.gr/maps/preview/place/Agathi+Beach,+Archaggelos+851+02/@36.176229,28.0983903,3221a,13.1y/data\\u003d!4m2!3m1!1s0x149572b1862de0c9:0xa545e1c8572377c2\



Anthony Quinn
https://www.google.gr/maps/preview/place/%CE%A0%CE%B1%CF%81%CE%B1%CE%BB%CE%AF%CE%B1+%CE%86%CE%BD%CF%84%CE%BF%CE%BD%CE%B9+%CE%9A%CE%BF%CF%85%CE%AF%CE%BD,+%CE%9A%CE%B1%CE%BB%CE%BB%CE%B9%CE%B8%CE%AD%CE%B1+851+00/@36.3214221,28.2093216,3215a,13.1y/data\\u003d!4m2!3m1!1s0x1495660b17c72697:0x3c5a25ec6368e6fc\



Faliraki
https://www.google.gr/maps/preview/place/%CE%A6%CE%B1%CE%BB%CE%B7%CF%81%CE%AC%CE%BA%CE%B9/@36.3349337,28.2077201,3214a,13.1y/data\\u003d!4m2!3m1!1s0x149565bb13071115:0x3e4786c4ec648968\



Fournoi
https://www.google.gr/maps/preview/place/%CE%A0%CE%B1%CF%81%CE%B1%CE%BB%CE%AF%CE%B1+%CE%A6%CE%BF

In [ ]:
def coords_from_link(link):
    coords_str = [p for p in link.split("/") if p[0] == "@"]
    if not coords_str:
        raise ValueError("Failed to find coordinates in {}.")
    if len(coords_str) > 1:
        raise ValueError("Found multiple coordinates in {}.".format(link))
    elif 
    
    